In [52]:
import os
os.listdir()

['.ipynb_checkpoints',
 '.jupyter',
 'bottom_market_cap.csv',
 'company_financial_health_scores.xlsx',
 'data.csv',
 'new_project_ao.ipynb',
 'top_company_shortlist.csv',
 'top_csv_financials_sortedbymarketcap.csv',
 'top_market_cap.csv',
 'Updated_Project_FIle.ipynb',
 'Updated_Project_FIle.ipynb.invalid']

In [81]:
import pandas as pd
df = pd.read_csv("data.csv",usecols=['ticker','name','total_score','total_grade', 'industry'])

In [82]:
df.columns

Index(['ticker', 'name', 'industry', 'total_score', 'total_grade'], dtype='object')

In [89]:
df['industry']

0                                   Media
1                             Automobiles
2      Trading Companies and Distributors
3                       Consumer products
4                                   Media
                      ...                
717                             Machinery
718                           Real Estate
719                            Technology
720                               Energy 
721                   Aerospace & Defense
Name: industry, Length: 722, dtype: object

In [50]:
df['total_score'].describe()

count     722.000000
mean      975.750693
std       218.751796
min       600.000000
25%       763.000000
50%      1046.000000
75%      1144.000000
max      1536.000000
Name: total_score, dtype: float64

In [51]:
df['total_grade'].value_counts()


total_grade
BBB    368
B      167
BB     104
A       83
Name: count, dtype: int64

In [143]:
#filtering top ESG companies
score_threshold = df['total_score'].median()
acceptable_grades = ["A","AA","AAA"]

df_high_esg = df[
    (df["total_score"] >= score_threshold) &
    (df["total_grade"].isin(acceptable_grades))
].copy()

df_high_esg_sorted = df_high_esg.sort_values(by="total_score",ascending=False)

# df_high_esg_sorted[["ticker","name","industry","total_score","total_grade"]]
df_high_esg_sorted.shape[0]

83

In [144]:
# Filtering lowest ESG companies
score_threshold = df['total_score'].median()
unacceptable_grades = ["B", "BB", "BBB", "CCC", "CC", "C", "D"] 

df_low_esg = df[
    (df["total_score"] < score_threshold) &
    (df["total_grade"].isin(unacceptable_grades))
].copy()

df_low_esg_sorted = df_low_esg.sort_values(by="total_score", ascending=True)
# Optional: View count or preview
# df_low_esg_sorted[["ticker", "name", "industry", "total_score", "total_grade"]]
# df_low_esg_sorted.shape[0]


In [134]:
import yfinance as yf

top_esg_tickers = df_high_esg_sorted["ticker"]

def get_financials(ticker):
    try:
        stock = yf.Ticker(ticker)
        info = stock.info
        return {
            "Ticker": ticker,
            "Sector": info.get("sector"),
            "marketCap": info.get("marketCap"),
            "Current Ratio": info.get("currentRatio"),
            "Quick Ratio": info.get("quickRatio"),
            "Debt to Equity": info.get("debtToEquity"),
            "Operating Margin": info.get("operatingMargins"),
            "Profit Margin": info.get("profitMargins"),
        }
    except Exception as e:
        print(f"Error fetching data for {ticker}: {e}")
        return {"ticker": ticker}

# Fetch financials for top ESG companies
financial_data = [get_financials(ticker) for ticker in top_esg_tickers]

# Create a DataFrame from the collected data
financials_df = pd.DataFrame(financial_data)
#sorted by market cap
df_sorted_by_mc = financials_df.sort_values(by="marketCap",ascending=False)

In [145]:
bottom_esg_tickers = df_low_esg_sorted["ticker"]
bottom_financial_data = [get_financials(ticker) for ticker in bottom_esg_tickers]
bottom_financial_df = pd.DataFrame(bottom_financial_data)
bottom_sorted_by_mc = bottom_financial_df.sort_values(by="marketCap", ascending=False)

HTTP Error 404: 
HTTP Error 404: 
HTTP Error 404: 
HTTP Error 404: 
HTTP Error 404: 
HTTP Error 404: 


In [146]:
# Optional preview
df_sorted_by_mc = df_sorted_by_mc.dropna() #drop rows with nulls
bottom_sorted_by_mc = bottom_sorted_by_mc.dropna()
bottom_sorted_by_mc

,Ticker,Sector,marketCap,Current Ratio,Quick Ratio,Debt to Equity,Operating Margin,Profit Margin
270,nvda,Technology,4.348063e+12,3.388,2.857,12.267,0.49108,0.51694
260,aapl,Technology,3.023880e+12,0.868,0.724,154.486,0.29991,0.24296
347,googl,Communication Services,2.359526e+12,1.904,1.720,11.481,0.32429,0.31118
70,meta,Communication Services,1.923747e+12,1.973,1.706,25.406,0.43019,0.39992
33,avgo,Technology,1.387594e+12,1.077,0.887,166.032,0.31765,0.22643
...,...,...,...,...,...,...,...,...
197,aemd,Healthcare,2.410369e+06,3.133,2.896,16.160,0.00000,0.00000
150,adtx,Healthcare,2.200433e+06,0.080,0.076,25.836,-5508.95360,0.00000
24,utrs,Healthcare,8.880000e+02,1.882,0.880,215.014,-0.50949,-0.64709
341,mtem,Healthcare,6.580000e+02,1.565,1.361,155.070,-14.50350,-0.61351


In [147]:
df_sorted_by_mc["Sector"].unique()

array(['Technology', 'Consumer Cyclical', 'Consumer Defensive',
       'Industrials', 'Healthcare', 'Utilities', 'Energy', 'Real Estate',
       'Basic Materials', 'Financial Services'], dtype=object)

In [148]:
industry_bench = {
    "Technology": {
        "profit_margin": 0.21,            # ~21% net margins for Info Tech sector :contentReference[oaicite:1]{index=1}
        "operating_margin": 0.21,
        "debt_to_equity": 0.65,           # typical IT range ~0.37‑0.92 :contentReference[oaicite:2]{index=2}
        "current_ratio": 2.0              # tech current ratio ~1.5‑3 :contentReference[oaicite:3]{index=3}
    },
    "Consumer Cyclical": {
        "profit_margin": 0.077,           # ~7.7% operating margin :contentReference[oaicite:4]{index=4}
        "operating_margin": 0.077,
        "debt_to_equity": 1.0,            # D/E typical range up to ~3.46 :contentReference[oaicite:5]{index=5}
        "current_ratio": 1.6              # retail & discretionary ~1.2‑1.8 :contentReference[oaicite:6]{index=6}
    },
    "Consumer Defensive": {
        "profit_margin": 0.069,           # consumer staples ~6.9% :contentReference[oaicite:7]{index=7}
        "operating_margin": 0.069,
        "debt_to_equity": 1.0,
        "current_ratio": 1.5
    },
    "Industrials": {
        "profit_margin": 0.106,           # industrials ~10.6% operating margin :contentReference[oaicite:8]{index=8}
        "operating_margin": 0.106,
        "debt_to_equity": 1.0,            # broad industrial range 0.28‑2.18 :contentReference[oaicite:9]{index=9}
        "current_ratio": 1.9              # industrials ~1.5‑2.4 :contentReference[oaicite:10]{index=10}
    },
    "Healthcare": {
        "profit_margin": 0.074,           # healthcare overall ~7.4% :contentReference[oaicite:11]{index=11}
        "operating_margin": 0.074,
        "debt_to_equity": 1.0,            # healthcare D/E ~0.53‑2.82 :contentReference[oaicite:12]{index=12}
        "current_ratio": 2.0              # typical ~1.5‑3 :contentReference[oaicite:13]{index=13}
    },
    "Utilities": {
        "profit_margin": 0.141,           # utilities net margins ~14.1% :contentReference[oaicite:14]{index=14}
        "operating_margin": 0.141,
        "debt_to_equity": 2.0,            # utilities often have high leverage ~1.08‑3.13 :contentReference[oaicite:15]{index=15}
        "current_ratio": 1.8              # utilities liquidity ~1.5‑2.0 :contentReference[oaicite:16]{index=16}
    },
    "Energy": {
        "profit_margin": 0.123,           # oil‑gas integrated ~12.3% :contentReference[oaicite:17]{index=17}
        "operating_margin": 0.123,
        "debt_to_equity": 1.3,            # energy D/E range ~0.29‑2.42 :contentReference[oaicite:18]{index=18}
        "current_ratio": 1.6              # capital‑intensive typical ~1.4‑2.0 :contentReference[oaicite:19]{index=19}
    },
    "Real Estate": {
        "profit_margin": 0.239,           # REIT net margin ~23.9% :contentReference[oaicite:20]{index=20}
        "operating_margin": 0.239,
        "debt_to_equity": 1.0,            # typical REIT ~0.54‑1.57 :contentReference[oaicite:21]{index=21}
        "current_ratio": 1.4              # REITs ~1.2‑1.6 :contentReference[oaicite:22]{index=22}
    },
    "Basic Materials": {
        "profit_margin": 0.101,           # materials ~10.1% operating profit margin :contentReference[oaicite:23]{index=23}
        "operating_margin": 0.101,
        "debt_to_equity": 0.9,            # materials D/E ~0.2‑1.29 :contentReference[oaicite:24]{index=24}
        "current_ratio": 1.8              # base materials ~1.6‑2.2 :contentReference[oaicite:25]{index=25}
    },
    "Financial Services": {
        "profit_margin": 0.135,           # non‑bank financial services ~13.5% :contentReference[oaicite:26]{index=26}
        "operating_margin": 0.135,
        "debt_to_equity": 2.0,            # financials up to ~4.5 but average ~2.0 :contentReference[oaicite:27]{index=27}
        "current_ratio": 1.5              # banks often around 1.3‑2.0 :contentReference[oaicite:28]{index=28}
    }
}

In [104]:
def score_ratio(company, benchmark, higher_higher=True):
    if company is None or pd.isna(company) or benchmark is None or pd.isna(benchmark):
        return 0
    if company == 0:
        return 0
    ratio = company / benchmark if higher_higher else benchmark / company
    return min(ratio, 2)


In [105]:
def calculate_scores(row):
    sector = row['Sector']
    if sector not in industry_bench:
        return pd.Series({
            'liquidity_score': 0,
            'solvency_score': 0,
            'op_margin_score': 0,
            'profit_margin_score': 0,
            'profit_score': 0,
            'financial_health_score': 0
        })

    bench = industry_bench[sector]

    liquidity_score = score_ratio(row['Current Ratio'], bench['current_ratio'])
    solvency_score = score_ratio(row['Debt to Equity'], bench['debt_to_equity'], higher_higher=False)
    op_margin_score = score_ratio(row['Operating Margin'], bench['operating_margin'])
    profit_margin_score = score_ratio(row['Profit Margin'], bench['profit_margin'])

    profit_score = (op_margin_score + profit_margin_score) / 2
    financial_health_score = (
        0.2 * liquidity_score +
        0.2 * solvency_score +
        0.6 * profit_score
    )

    return pd.Series({
        'liquidity_score': liquidity_score,
        'solvency_score': solvency_score,
        'op_margin_score': op_margin_score,
        'profit_margin_score': profit_margin_score,
        'profit_score': profit_score,
        'financial_health_score': financial_health_score
    })


In [106]:
df_sorted_by_mc = df_sorted_by_mc.join(df_sorted_by_mc.apply(calculate_scores, axis=1))

In [149]:
bottom_sorted_by_mc = bottom_sorted_by_mc.join(bottom_sorted_by_mc.apply(calculate_scores, axis=1))

In [150]:
bottom_sorted_by_mc

,Ticker,Sector,marketCap,Current Ratio,Quick Ratio,Debt to Equity,Operating Margin,Profit Margin,liquidity_score,solvency_score,op_margin_score,profit_margin_score,profit_score,financial_health_score
270,nvda,Technology,4.348063e+12,3.388,2.857,12.267,0.49108,0.51694,1.6940,0.052988,2.000000,2.000000,2.000000,1.549398
260,aapl,Technology,3.023880e+12,0.868,0.724,154.486,0.29991,0.24296,0.4340,0.004208,1.428143,1.156952,1.292548,0.863170
347,googl,Communication Services,2.359526e+12,1.904,1.720,11.481,0.32429,0.31118,0.0000,0.000000,0.000000,0.000000,0.000000,0.000000
70,meta,Communication Services,1.923747e+12,1.973,1.706,25.406,0.43019,0.39992,0.0000,0.000000,0.000000,0.000000,0.000000,0.000000
33,avgo,Technology,1.387594e+12,1.077,0.887,166.032,0.31765,0.22643,0.5385,0.003915,1.512619,1.078238,1.295429,0.885740
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
197,aemd,Healthcare,2.410369e+06,3.133,2.896,16.160,0.00000,0.00000,1.5665,0.061881,0.000000,0.000000,0.000000,0.325676
150,adtx,Healthcare,2.200433e+06,0.080,0.076,25.836,-5508.95360,0.00000,0.0400,0.038706,-74445.318919,0.000000,-37222.659459,-22333.579935
24,utrs,Healthcare,8.880000e+02,1.882,0.880,215.014,-0.50949,-0.64709,0.9410,0.004651,-6.885000,-8.744459,-7.814730,-4.499708
341,mtem,Healthcare,6.580000e+02,1.565,1.361,155.070,-14.50350,-0.61351,0.7825,0.006449,-195.993243,-8.290676,-102.141959,-61.127386


In [118]:
# Merge ESG data (on Ticker)
final_esg_mc_df = df_sorted_by_mc[["Ticker", "Sector", "marketCap", "financial_health_score"]].merge(df[['ticker','name','total_score','total_grade']], left_on="Ticker", right_on="ticker",
                                                                                                     how="left")

# Assign Rank based on marketCap (already sorted)
final_esg_mc_df = final_esg_mc_df.reset_index(drop=True)
final_esg_mc_df["Rank"] = final_esg_mc_df.index + 1

# Reorder columns for PowerBI
cols = ["Rank", "Ticker", "name", "Sector", "marketCap", "total_score", "total_grade", "financial_health_score"]
final_esg_mc_df = final_esg_mc_df[cols]


In [151]:
# Merge ESG data (on Ticker)
final_bottom_esg_mc_df = bottom_sorted_by_mc[["Ticker", "Sector", "marketCap", "financial_health_score"]].merge(df[['ticker','name','total_score','total_grade']], left_on="Ticker", right_on="ticker",
                                                                                                     how="left")

# Assign Rank based on marketCap (already sorted)
final_bottom_esg_mc_df = final_bottom_esg_mc_df.reset_index(drop=True)
final_bottom_esg_mc_df["Rank"] = final_bottom_esg_mc_df.index + 1

# Reorder columns for PowerBI
cols = ["Rank", "Ticker", "name", "Sector", "marketCap", "total_score", "total_grade", "financial_health_score"]
final_bottom_esg_mc_df = final_bottom_esg_mc_df[cols]


In [123]:
# Cap the financial health score between -2 and 2
final_esg_mc_df["financial_health_score_clipped"] = final_esg_mc_df["financial_health_score"].clip(lower=-2, upper=2)

In [152]:
final_bottom_esg_mc_df["financial_health_score_clipped"] = final_bottom_esg_mc_df["financial_health_score"].clip(lower=-2, upper=2)

In [153]:
final_bottom_esg_mc_df.to_csv("bottom_ranked_by_marketcap_with_esg.csv", index=False)

In [125]:
# Export to CSV
final_esg_mc_df.to_csv("ranked_by_marketcap_with_esg.csv", index=False)

In [154]:
final_bottom_esg_mc_df

,Rank,Ticker,name,Sector,marketCap,total_score,total_grade,financial_health_score,financial_health_score_clipped
0,1,nvda,NVIDIA Corp,Technology,4.348063e+12,899,BB,1.549398,1.549398
1,2,aapl,Apple Inc,Technology,3.023880e+12,891,BB,0.863170,0.863170
2,3,googl,Alphabet Inc,Communication Services,2.359526e+12,1012,BBB,0.000000,0.000000
3,4,meta,Meta Platforms Inc,Communication Services,1.923747e+12,652,B,0.000000,0.000000
4,5,avgo,Broadcom Inc,Technology,1.387594e+12,622,B,0.885740,0.885740
...,...,...,...,...,...,...,...,...,...
232,233,aemd,Aethlon Medical Inc,Healthcare,2.410369e+06,777,BB,0.325676,0.325676
233,234,adtx,Aditxt Inc,Healthcare,2.200433e+06,739,B,-22333.579935,-2.000000
234,235,utrs,Minerva Surgical Inc,Healthcare,8.880000e+02,616,B,-4.499708,-2.000000
235,236,mtem,Molecular Templates Inc,Healthcare,6.580000e+02,1005,BBB,-61.127386,-2.000000
